In [ ]:
from jupyter_dash import JupyterDash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output

import pandas as pd
import plotly.graph_objects as go

# DATA PREPROCESSING ==========================================
df = pd.read_csv("nakazeni-vyleceni-umrti-testy.csv")

df.set_index(['datum'], drop = True, inplace = True)
df.index = pd.to_datetime(df.index)
new_names = ["inf", "cur", "dead", "tests", "ag", "d_inf", "d_cur", "d_dead", "d_tests", "d_ag"]

df.columns = new_names

df = df[["inf", "dead", "d_inf", "d_dead"]]
pretty_names = {
    "inf" : "Nakažení celkem",
    "dead": "Mrtví celkem",
    "d_inf" : "Přírůstek nakažených",
    "d_dead" : "Přírůstek mrtvých"
}

aggr = {"inf" : "sum", "dead" : "sum", "d_inf" : "mean", "d_dead" : "mean"}
# ==============================================================

app = JupyterDash(__name__)


# priprava aktivnich prvku bokem - app layout je pak trochu prehlednejsi

theme_dropdown = dcc.Dropdown(
            id='theme-dropdown', clearable=False,
            value='plotly_white',
            options=[{'label': c, 'value': c} for c in ["plotly_white", "plotly_dark"]]
            )

data_checklist = dcc.Checklist(
            id='data-checklist',
            value=['d_dead'],
            options=[{'label': pretty_names[d], 'value': d} for d in ["d_inf", "d_dead"]]
        )

num_input = dcc.Input(
            id="days", type="number", value = 1,
            min=1, max=10, step=1,
        )

data_checklist_cum = dcc.Checklist(
            id='data-cummulative',
            value=['dead'],
            options=[{'label': pretty_names[d], 'value': d} for d in ["inf", "dead"]],
            )

log_check = dcc.Checklist(
        id='logscale',
        value = [],
        options=[{'label': "Logaritmická škála", 'value': "logscale_y"}],
    )

# samotny layout ===================================================

app.layout = html.Div([
    html.H1("Covid"),
    html.Label(["theme", theme_dropdown]),
    html.Label(["Data", data_checklist]),
    html.Label(["Klouzavý průměr ", num_input]),
    dcc.Graph(id="graph"),
    html.Hr(),
    html.Label(["Data", data_checklist_cum, log_check]),
    dcc.Graph(id="graph2"),
])

# callbacky =========================================================

@app.callback(
    Output("graph", "figure"),
    Input("theme-dropdown", "value"),
    Input("data-checklist", "value"),
    Input("days", "value"))
def update_figure1(theme, data, days):
    if days is None:
        days = 1
    fig = go.Figure()
    for dt in data:
        fig.add_trace(go.Scatter(x = df.index, y = df[dt], name = pretty_names[dt]))

    if days > 1:
        resampled = df.resample("%dD" % days).agg(aggr)
        for dt in data:
            fig.add_trace(go.Scatter(x = resampled.index, y = resampled[dt],
                                     name = pretty_names[dt] + f" {days}D avg"))
        
    fig.update_layout(template = theme, title = "COVID19 - Přírůstky")
    return fig


@app.callback(
    Output("graph2", "figure"),
    Input("theme-dropdown", "value"),
    Input("data-cummulative", "value"),
    Input("logscale", "value"))
def update_figure2(theme, data, logscale):
    if len(logscale) == 0:
        logscale = False
    fig = go.Figure()

    for dt in data:
        fig.add_trace(go.Scatter(x = df.index, y = df[dt], name = pretty_names[dt]))
    
    if logscale:
        fig.update_yaxes(type="log")    
    fig.update_layout(template = theme, title = "COVID19 - Kumulativní počty")
    return fig

app.run_server(mode="inline")